In [ ]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env file - NOTE: PLEASE SET YOUR OWN .env FILE BASED ON .env.example

from datasets import load_from_disk, Dataset
from copy import deepcopy
from tqdm.notebook import tqdm

from google import genai
from pydantic import BaseModel
import os

import json
import time

import pickle
import re

In [4]:
GOOGLE_GEMINI_API_KEY = os.getenv("GOOGLE_GEMINI_API_KEY", "")
GEMINI_MODEL = os.getenv("GEMINI_MODEL", "")

In [5]:
# Load dataset from disk
custom_dataset = load_from_disk("./mbpp_preprocessed_dataset")

In [9]:
class DPOPair(BaseModel):
    task_id: int
    code_output_1: str
    code_output_2: str

In [10]:
client = genai.Client(api_key=GOOGLE_GEMINI_API_KEY)

In [ ]:
# Create inline requests for each example in the dataset
inline_requests = [
    {
        'contents': [{
            'parts': [{'text': "Task_ID:" + str(example["task_id"]) + "\n\n" + "Create 2 different implementations of the following function as specified in the following requirements. Ensure that both implementations are functionally correct, but differ in their approach and structure. Follow Python best practices.\n\n" + example["text"]}],
            'role': 'user'
        }],
        'config': {
            'response_mime_type': 'application/json',
            'response_schema': DPOPair
        }
    } for example in custom_dataset
]


In [ ]:
# Create batch job
inline_batch_job = client.batches.create(
    model=GEMINI_MODEL,
    src=inline_requests,
    config={
        'display_name': "structured-output-job-1"
    },
)

In [ ]:
# View batch job name for easy access
inline_batch_job.name

'batches/8kihianc0l465o17ga043cliqjfa1nt0h24b'

In [22]:
# wait for the job to finish
job_name = inline_batch_job.name
print(f"Polling status for job: {job_name}")

while True:
    batch_job_inline = client.batches.get(name=job_name)
    if batch_job_inline.state.name in ('JOB_STATE_SUCCEEDED', 'JOB_STATE_FAILED', 'JOB_STATE_CANCELLED', 'JOB_STATE_EXPIRED'):
        break
    print(f"Job not finished. Current state: {batch_job_inline.state.name}. Waiting 30 seconds...")
    time.sleep(30)

print(f"Job finished with state: {batch_job_inline.state.name}")

Polling status for job: batches/8kihianc0l465o17ga043cliqjfa1nt0h24b
Job not finished. Current state: JOB_STATE_RUNNING. Waiting 30 seconds...
Job not finished. Current state: JOB_STATE_RUNNING. Waiting 30 seconds...
Job not finished. Current state: JOB_STATE_RUNNING. Waiting 30 seconds...
Job not finished. Current state: JOB_STATE_RUNNING. Waiting 30 seconds...
Job not finished. Current state: JOB_STATE_RUNNING. Waiting 30 seconds...
Job not finished. Current state: JOB_STATE_RUNNING. Waiting 30 seconds...
Job not finished. Current state: JOB_STATE_RUNNING. Waiting 30 seconds...
Job not finished. Current state: JOB_STATE_RUNNING. Waiting 30 seconds...
Job not finished. Current state: JOB_STATE_RUNNING. Waiting 30 seconds...
Job not finished. Current state: JOB_STATE_RUNNING. Waiting 30 seconds...
Job not finished. Current state: JOB_STATE_RUNNING. Waiting 30 seconds...
Job not finished. Current state: JOB_STATE_RUNNING. Waiting 30 seconds...
Job not finished. Current state: JOB_STATE_

In [ ]:
batch_job = client.batches.get(name=job_name)

In [ ]:
# Retrieve results
if batch_job.state.name == 'JOB_STATE_SUCCEEDED':
    # If batch job was created with a file
    if batch_job.dest and batch_job.dest.file_name:
        # Results are in a file
        result_file_name = batch_job.dest.file_name
        print(f"Results are in file: {result_file_name}")

        print("Downloading result file content...")
        file_content = client.files.download(file=result_file_name)
        # Process file_content (bytes) as needed
        print(file_content.decode('utf-8'))

    # If batch job was created with inline request
    # (for embeddings, use batch_job.dest.inlined_embed_content_responses)
    elif batch_job.dest and batch_job.dest.inlined_responses:
        # Results are inline
        print("Results are inline:")
        for i, inline_response in enumerate(batch_job.dest.inlined_responses):
            print(f"Response {i+1}:")
            if inline_response.response:
                # Accessing response, structure may vary.
                try:
                    print(inline_response.response.text)
                except AttributeError:
                    print(inline_response.response) # Fallback
            elif inline_response.error:
                print(f"Error: {inline_response.error}")
    else:
        print("No results found (neither file nor inline).")
else:
    print(f"Job did not succeed. Final state: {batch_job.state.name}")
    if batch_job.error:
        print(f"Error: {batch_job.error}")

Results are inline:
Response 1:
{
  "task_id": 602,
  "code_output_1": "def first_repeated_char(str1):\n    \"\"\"\n    Finds the first repeated character in a string using a set to track seen characters.\n\n    This approach iterates through the string once, offering O(n) time complexity,\n    where n is the length of the string. The space complexity is O(k), where k is\n    the number of unique characters.\n\n    Args:\n        str1: The input string.\n\n    Returns:\n        The first character that is repeated, or None if no character is repeated.\n    \"\"\"\n    seen_chars = set()\n    for char in str1:\n        if char in seen_chars:\n            return char\n        seen_chars.add(char)\n    return None",
  "code_output_2": "def first_repeated_char(str1):\n    \"\"\"\n    Finds the first repeated character by checking indices of first occurrences.\n\n    This approach iterates through the string and for each character, it searches\n    the string again from the beginning. This 

In [ ]:
# Save raw inline responses to a pickle file for safekeeping
with open("gemini_raw_response.pkl", "wb") as f:
    pickle.dump(batch_job.dest.inlined_responses, f)

In [ ]:
# Load raw inline responses from pickle file
with open("gemini_raw_response.pkl", "rb") as f:
    responses = pickle.load(f)

In [ ]:
# Print loaded responses
for i, inline_response in enumerate(responses):
    # print(f"Response {i+1}:")
    if inline_response.response:
        # Accessing response, structure may vary.
        try:
            print(inline_response.response.text)
        except AttributeError:
            print(inline_response.response) # Fallback
    elif inline_response.error:
        print(f"Error: {inline_response.error}")

{
  "task_id": 602,
  "code_output_1": "def first_repeated_char(str1):\n    \"\"\"\n    Finds the first repeated character in a string using a set to track seen characters.\n\n    This approach iterates through the string once, offering O(n) time complexity,\n    where n is the length of the string. The space complexity is O(k), where k is\n    the number of unique characters.\n\n    Args:\n        str1: The input string.\n\n    Returns:\n        The first character that is repeated, or None if no character is repeated.\n    \"\"\"\n    seen_chars = set()\n    for char in str1:\n        if char in seen_chars:\n            return char\n        seen_chars.add(char)\n    return None",
  "code_output_2": "def first_repeated_char(str1):\n    \"\"\"\n    Finds the first repeated character by checking indices of first occurrences.\n\n    This approach iterates through the string and for each character, it searches\n    the string again from the beginning. This results in a time complexity of\

In [35]:
# Processes the inline responses into JSON
dpopair_json = [json.loads(inline_response.response.text) for inline_response in responses]

# Save the processed JSON data to a file
with open("dpopair_json.json", "w") as f:
    json.dump(dpopair_json, f, indent=2)

In [37]:
# Restructure the JSON data for easier concatenation
# e.g. {[task_id: {code_output_1: ..., code_output_2: ...}, ...]}
# Alternative approach, dictionary comprehension
restructured_dpopair_json = {
    pair['task_id']: {
        "code_output_1": pair['code_output_1'],
        "code_output_2": pair['code_output_2']
    }
    for pair in dpopair_json
}

In [ ]:
# Add the code outputs back into the dataset
custom_dataset_with_responses = deepcopy(custom_dataset)
custom_dataset_with_responses = custom_dataset_with_responses.to_list()

for i, example in enumerate(custom_dataset_with_responses):
    task_id = example['task_id']
    custom_dataset_with_responses[i]['code_output_1'] = restructured_dpopair_json[task_id]['code_output_1']
    custom_dataset_with_responses[i]['code_output_2'] = restructured_dpopair_json[task_id]['code_output_2']

custom_dataset_with_responses = Dataset.from_list(custom_dataset_with_responses)

In [40]:
custom_dataset_with_responses[0]

{'task_id': 602,
 'text': 'Write a python function to find the first repeated character in a given string.\nThe main function is defined by the function signature: def first_repeated_char(str1)\n',
 'code': 'def first_repeated_char(str1):\r\n  for index,c in enumerate(str1):\r\n    if str1[:index+1].count(c) > 1:\r\n      return c \r\n  return "None"',
 'test_list': ['assert first_repeated_char("abcabc") == "a"',
  'assert first_repeated_char("abc") == "None"',
  'assert first_repeated_char("123123") == "1"'],
 'test_setup_code': '',
 'challenge_test_list': [],
 'code_output_1': 'def first_repeated_char(str1):\n    """\n    Finds the first repeated character in a string using a set to track seen characters.\n\n    This approach iterates through the string once, offering O(n) time complexity,\n    where n is the length of the string. The space complexity is O(k), where k is\n    the number of unique characters.\n\n    Args:\n        str1: The input string.\n\n    Returns:\n        The f

In [56]:
def test_one_example(example, code_key='code', test_list_key='test_list'):
    temp_namespace = {}

    try:
        exec(example[code_key], temp_namespace)
    except SyntaxError as e:
        print(f"SyntaxError in code for example id: {example['task_id']}")
        print(f"  -> {e}")
        del temp_namespace
        return False

    print(f"Running tests for example id: {example['task_id']}")

    all_tests_passed = True
    for i, test in enumerate(example[test_list_key]):
        try:
            exec(test, temp_namespace)
            # print(f"Test {i+1} passed.")
        except AssertionError as e:
            all_tests_passed = False
            print(f"Test {i+1} failed.")
            print(f"  -> AssertionError: {e}")
        except Exception as e:
            all_tests_passed = False
            print(f"Test {i+1} failed.")
            print(f"  -> Exception: {e}")
    
    del temp_namespace

    return all_tests_passed

In [ ]:
# Function to automatically verify all examples in the dataset against their unit test cases
def test_all_examples(code_dataset, code_key='code', test_list_key='test_list'):
    passed = []
    failed = []
    for i, example in tqdm(enumerate(code_dataset)):
        all_tests_passed = test_one_example(example, code_key=code_key, test_list_key=test_list_key)
        if not all_tests_passed:
            print(f"Some tests failed for example id: {example['task_id']}")
            failed.append(example['task_id'])
        else:
            passed.append(example['task_id'])
    print(f"Total examples: {len(code_dataset)} - Passed: {len(passed)}, Failed: {len(failed)}")
    return passed, failed

In [ ]:
# Evalate the first code output of each example
passed_1, failed_1 = test_all_examples(custom_dataset_with_responses, code_key='code_output_1', test_list_key='test_list')

0it [00:00, ?it/s]

Running tests for example id: 602
Test 2 failed.
  -> AssertionError: 
Some tests failed for example id: 602
Running tests for example id: 603
Running tests for example id: 604
Running tests for example id: 605
Running tests for example id: 606
Running tests for example id: 607
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Some tests failed for example id: 607
Running tests for example id: 608
Running tests for example id: 609
Test 1 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Some tests failed for example id: 609
Running tests for example id: 610
Running tests for example id: 611
Running tests for example id: 612
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Some tests failed for example id: 612
Running tests for example id: 613
Test 1 failed.
  -> Exception: cannot unpack non-iterable int object
Test 2 failed.
  -> Exception: cannot 

<string>:7: SyntaxWarning: invalid escape sequence '\s'


In [ ]:
# Evaluate the second code output of each example
passed_2, failed_2 = test_all_examples(custom_dataset_with_responses, code_key='code_output_2', test_list_key='test_list')

0it [00:00, ?it/s]

Running tests for example id: 602
Test 2 failed.
  -> AssertionError: 
Some tests failed for example id: 602
Running tests for example id: 603
Running tests for example id: 604
Running tests for example id: 605
Running tests for example id: 606
Running tests for example id: 607
Test 1 failed.
  -> Exception: search() got an unexpected keyword argument 'pos'
Test 2 failed.
  -> Exception: search() got an unexpected keyword argument 'pos'
Test 3 failed.
  -> Exception: search() got an unexpected keyword argument 'pos'
Some tests failed for example id: 607
Running tests for example id: 608
Running tests for example id: 609
Test 1 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Some tests failed for example id: 609
Running tests for example id: 610
Running tests for example id: 611
Running tests for example id: 612
Test 1 failed.
  -> AssertionError: 
Test 2 failed.
  -> AssertionError: 
Test 3 failed.
  -> AssertionError: 
Some tests failed for example id: 612
Running t

<string>:8: SyntaxWarning: invalid escape sequence '\s'
<string>:8: SyntaxWarning: invalid escape sequence '\d'


In [ ]:
# Find examples where both implementations passed all tests
collectively_correct = set(passed_1).intersection(set(passed_2))

In [ ]:
# Create a new dataset with only the collectively correct examples
custom_dataset_with_both_correct = Dataset.from_list([example for example in custom_dataset_with_responses if example['task_id'] in collectively_correct])

In [110]:
# Save the dataset with both implementations correct to disk
custom_dataset_with_both_correct.save_to_disk("./mbpp_custom_dataset_with_correct_gemini_pairs")

Saving the dataset (0/1 shards):   0%|          | 0/238 [00:00<?, ? examples/s]